<img src="logo_quadri.jpg">


# <center>Informatique tc1 - TD3 Compression d'images</center>

## <center style="color: #66d">Pattée Armand - Dalbiès Claire</center>

### <center>Groupe A2a - Professeur encadrant : Liming Chen</center>

## Partie 1 : Chargement d'une image et fonctions utilitaires

### Ex 1.1

In [1]:
from PIL import Image
from math import sqrt, log10
im=Image.open("roger.jpg")
px=im.load()
W,H=im.size
seuil=10

def peindre_rect_couleur(x0,y0,L,l,r,v,b):
    for i in range(x0,x0+L):
        for j in range(y0,y0+l):
            px[i,j]=int(r),int(v),int(b)
    #return im.show() : on le met en commentaire car cela pose des problèmes pour la suite

In [2]:
peindre_rect_couleur(650,76,190,100,25,120,7)
 #rectangle de couleur verte

Le résultat donne l'image suivante : 

<img src="capture1.png">

In [3]:
peindre_rect_couleur(0,0,W,H,100,25,120) #rectangle violet des dimensions de l'image

L'essai précédent doit repeindre toute l'image en violet, comme le montre la figure ci-dessous : 
<img src="capture2.png">   

### Ex 1.2

In [4]:
im=Image.open("roger.jpg")
px=im.load()

def moyenne(x0,y0,L,l):
    N=L*l
    R=0
    V=0
    B=0
    for i in range(x0,x0+L):
        for j in range(y0,y0+l):
            R+=(px[i,j])[0]
            V+=(px[i,j])[1]
            B+=(px[i,j])[2]
    return(R/N,V/N,B/N)

In [5]:
moyenne(650,76,190,100) #test sur la photo de Roger Federer sur la même zone qui a précédemment été colorée en vert

(23.143473684210527, 24.472315789473683, 23.628105263157895)

On obtient une couleur moyenne qui correspond à une teinte de noir, ce qui est cohérent avec la zone choisie (plutôt sombre).

### Ex 1.3

In [6]:
def ecart_type(x0,L,y0,l):
    MR,MV,MB=moyenne(x0,y0,L,l)
    N=L*l
    SR=0
    SV=0
    SB=0
    for i in range (x0,x0+L):
        for j in range(y0,y0+l):
            SR+=(px[i,j])[0]**2
            SV+=(px[i,j])[1]**2
            SB+=(px[i,j])[2]**2
    return (sqrt(SR/N-MR**2),sqrt(SV/N-MV**2),sqrt(SB/N-MB**2))


In [7]:
ecart_type(650,190,76,100) #test sur l'écart-type


(7.191518728177551, 7.678135183771974, 7.124406500819532)

Du fait de l'homogénéité de la zone retenue, l'écart type des différentes couleur est assez faible.

### Ex 1.4

In [8]:
def homogene(x0,y0,L,l,sig):
    sr=0
    sg=0
    sb=0
    ssr=0
    ssg=0
    ssb=0
    for i in range(x0,x0+L):
        for j in range(y0,y0+l):
            a,b,c = px[i,j]
            sr+=a
            sg+=b
            sb+=c
            ssr+=a*a
            ssg+=b*b
            ssb+=c*c
    if L*l!=0:
        n=L*l
    else:
        n=1
    mr=sr/n
    mg=sg/n
    mb=sb/n
    
    ecr=sqrt(ssr/n -mr**2)
    ecg=sqrt(ssg/n -mg**2)
    ecb=sqrt(ssb/n -mb**2)
    
    if ecr>sig or ecg>sig or ecb>sig :
        return(int(mr),int(mg),int(mb),(ecr+ecg+ecb)/3,False)
    return(int(mr),int(mg),int(mb),(ecr+ecg+ecb)/3,True)

In [9]:
homogene(650,76,190,100,seuil) #zone choisie est essentiellement constituée de pixels noirs 

(23, 24, 23, 7.331353470923019, True)

La fonction homogene nous dit si la zone choisie est homogène ou non en prenant un certain seuil en entrée. Cette fonction retourne les valeurs moyennes du rouge, du bleu et du vert ainsi que la moyenne de leurs écarts-type tout en indiquant par "True" ou "None" si la zone est homogène. Cette fonction va nous être utile pour la suite du TD.

### Ex 1.5

In [10]:
def copie_matrice(px,x0,L,y0,l):
    res=[[0 for y in range(y0,y0+l)]for x in range(x0,x0+L)]
    for x in range(x0,x0+L):
        for y in range (y0,y0+l):
            res[x-x0][y-y0]=px[x,y]
    return res

def division(mat):
    w=len(mat)
    h=len(mat[0])
    if w>1 and h>1: #type0
        if w%2==0 and h%2==0:
            return[copie_matrice(mat,0,0,w/2,h/2),\
                   copie_matrice(mat,w/2,0,w,h/2),\
                   copie_matrice(mat,0,h/2,w/2,h),\
                   copie_matrice(mat,w/2,h/2,w,h)]
        elif w%2==0 and h%2==1:
            return[copie_matrice(mat,0,0,w/2,h/2+1/2),\
                   copie_matrice(mat,w/2,0,w,h/2+1/2),\
                   copie_matrice(mat,0,h/2+1/2,w/2,h),\
                   copie_matrice(mat,w/2,h/2+1/2,w,h)]
        elif w%2==1 and h%2==0:
            return[copie_matrice(mat,0,0,w/2+1/2,h/2),\
                   copie_matrice(mat,w/2+1/2,0,w,h/2),\
                   copie_matrice(mat,0,h/2,w/2+1/2,h),\
                   copie_matrice(mat,w/2+1/2,h/2,w,h)]
        else:
            return[copie_matrice(mat,0,0,w/2+1/2,h/2+1/2),\
                   copie_matrice(mat,w/2+1/2,0,w,h/2+1/2),\
                   copie_matrice(mat,0,h/2+1/2,w/2+1/2,h),\
                   copie_matrice(mat,w/2+1/2,h/2+1/2,w,h)]
    else:
        if w==1 and h>1:
            if h%4==0: #type9
                return mat[[mat[0][y] for y in range(0,h/4)],[mat[0][y] for y in range(h/4,h/2)],[mat[0][y] for y in range(h/2,3*h/4)],[mat[0][y] for y in range(3*h/4,h)]]
            elif h%2==0: #type2
                return mat[[mat[0][y] for y in range(0,h/2)],[mat[0][y] for y in range(h/2,h)]]
            else: #type2
                return mat[[mat[0][y] for y in range(0,h/2+1/2)],[mat[0][y] for y in range(h/2+1/2,h)]]
        elif h==1 and w>1:
            if w%4==0: #type8
                return mat[[mat[x][0] for x in range(0,w/4)],[mat[x][0] for x in range(w/4,w/2)],[mat[x][0] for x in range(w/2,3*w/4)],[mat[x][0] for x in range(3*w/4,w)]]
            if w%2==0: #type1
                return mat[[mat[x][0] for x in range(0,w/2)],[mat[x][0] for x in range(w/2,w)]]
            else: #type1
                return mat[[mat[x][0] for x in range(0,w/2+1/2)],[mat[x][0] for x in range(w/2+1/2,w)]]
        elif w==h==1: #type3
            return mat


Tous les cas (dimensions paires et impaires) sont traités : 
- les cas où la région est d'une aire supérieure à 4
- les cas où la région est d'une aire inférieure à 4  
- le cas où l'aire est égale à 1

## Partie 2 : Création d'arbre explicite et parcours

### Ex 2.1 et 2.2

In [11]:
class Node :
    def __init__ (self, x1, y1, w, h, im, s) :
        self.__x1 = x1
        self.__y1 = y1
        self.__w = w
        self.__h = h
        self.__fils = []
        self.__im = im
        self.__seuil = s
        d = division(x1,y1,w,h)
        n = len(d)
        if not self.est_terminal() :
            for k in range(n):
                x1, y1, w, h = d[k]
                self.__fils += [Node(x1,y1,w,h,im,s)]
                
    def est_terminal(self) :
        return homogene(self.__x1, self.__w, self.__y1, self.__h)
        
    def __repr__ (self) : #surcharger la fonction print (q 2.2)
        txt = " x1 = " + str(self.__x1) + ", y1 = " + str(self.__y1) + ", w = " + str(self.__w) + ", h = " + str(self.__h) + ", seuil = " + str(self.__seuil)
        k = 1
        for f in self.__fils :
            txt += '\n' + "Fils " + str(k)
            txt += '\n' + f.__str__()
            k+=1
            
        return txt
    
    def get_dim (self) :
        return self.__x1, self.__w, self.__y1, self.__h
    
    def get_fils (self) :
        return self.__fils



On a choisi de créer certaines fonctions comme get_fils ou get_dim qui permettent d'avoir accès directement aux fils ou aux dimensions.

### Ex 2.3

In [14]:
def arbre(x0,y0,L,l,t,i=0):
    if L*l>=4:
        a,b,d,e,flag=homogene(x0,y0,L,l,seuil)
        if e>=seuil:
            t.append([x0,y0,L,l,None,i])
            arbre(x0,y0,L//2,l//2,t,i+1)
            arbre(int(x0+L//2),y0,L//2+L%2,l//2,t,i+1)
            arbre(x0,int(y0+l//2),L//2,l//2+l%2,t,i+1)
            arbre(int(x0+L//2),int(y0+l//2),L//2+L%2,l//2+l%2,t,i+1)
    else :
            t.append([x0,y0,L,l,[moyenne(x0,y0,L,l)],i])

In [15]:
t=[]
arbre(0,0,W,H,t,i=0) #test sur la totalité de l'image


In [16]:
print(t)

[[0, 0, 848, 565, None, 0], [0, 0, 424, 282, None, 1], [0, 0, 212, 141, None, 2], [0, 0, 106, 70, None, 3], [0, 0, 53, 35, None, 4], [0, 0, 26, 17, None, 5], [13, 8, 13, 9, None, 6], [13, 12, 6, 5, None, 7], [16, 14, 3, 3, None, 8], [16, 14, 1, 1, [(41.0, 47.0, 45.0)], 9], [17, 14, 2, 1, [(52.0, 58.0, 57.0)], 9], [16, 15, 1, 2, [(52.5, 57.5, 61.0)], 9], [19, 12, 7, 5, None, 7], [19, 14, 3, 3, None, 8], [19, 14, 1, 1, [(61.0, 67.0, 67.0)], 9], [20, 14, 2, 1, [(69.5, 74.5, 77.5)], 9], [19, 15, 1, 2, [(77.0, 82.0, 86.5)], 9], [22, 14, 4, 3, None, 8], [22, 14, 2, 1, [(86.0, 91.0, 96.0)], 9], [24, 14, 2, 1, [(97.5, 101.5, 111.5)], 9], [26, 0, 27, 17, None, 5], [26, 0, 13, 8, None, 6], [32, 4, 7, 4, None, 7], [39, 0, 14, 8, None, 6], [46, 0, 7, 4, None, 7], [39, 4, 7, 4, None, 7], [46, 4, 7, 4, None, 7], [26, 8, 13, 9, None, 6], [26, 8, 6, 4, None, 7], [32, 8, 7, 4, None, 7], [26, 12, 6, 5, None, 7], [26, 14, 3, 3, None, 8], [26, 14, 1, 1, [(108.0, 112.0, 123.0)], 9], [27, 14, 2, 1, [(117.5,

On crée une fonction récursive avec une liste. On stocke dans la liste des listes contenant : la position du rectangle, les dimensions les couleurs ainsi que la profondeur de ce carré dans l’arbre (par le biais de i) ce qui nous sera utile pour les questions suivantes. On met un None à la place des couleurs s’il ne s’agit pas d’une racine : on peut alors savoir si on est sur un neud terminal ou non.

### Ex 2.4

In [17]:
def peindreArbre(t):
    for i in range (len(t)-1):
        if t[i][4]!=None:
            peindre_rect_couleur(t[i][0],t[i][1],t[i][2],t[i][3],t[i][4][0][0],t[i][4][0][1],t[i][4][0][2])
    return im.show()
            

On crée une fonction récursive qui parcourt l'arbre depuis sa racine, et peint chaque noeud terminal selon la couleur moyenne de ses pixels dans l'arbre.

In [18]:
peindreArbre(t)

La figure ci-dessous donne le résultat pour l'image de Roger Federer. On remarque que l'image est déjà moins nette (gauche), elle a bien été compressée.
<img src="peindrearbre.png">  

### Ex 2.5

In [19]:
def peindreProfondeur(t):
    for i in range (len(t)-1):
        if t[i][4]!=None:
            peindre_rect_couleur(t[i][0],t[i][1],t[i][2],t[i][3],255-20*t[i][5],255-20*t[i][5],255-20*t[i][5])
    return im.show()
    
    

On crée ici une fonction récursive qui peint les noeuds terminaux d'un arbre d'une couleur proportionnelle à leur profondeur dans l'arbre.
R,V,B=0,0,0 donne du blanc donc on ajoute des valeurs qui dépendent des valeurs de i[5] (la
profondeur) ici 20*i[5] pour être sûr de bien voir la différence entre les teintes de noir pour chaque
profondeur.

In [20]:
peindreProfondeur(t)

On obtient alors la figure suivante : 
<img src="peindre2.png">

### Ex 2.6

In [21]:
def PSNR():
    imBase = Image.open('roger.jpg')
    pxBase = imBase.load()
    px=im.load()
    s=0
    for i in range(W):
        for j in range(H):
            s+=(px[i,j][0]-pxBase[i,j][0])**2+(px[i,j][1]-pxBase[i,j][1])**2+(px[i,j][1]-pxBase[i,j][1])**2
    s=20*log10(255)-10*log10(s/(3*W*H))
    return(s)

In [22]:
s=PSNR()
print(s)

19.166491089260802


On devait normalement estimer l'efficacité de notre code grâce à cette valeur, mais le site indiqué sur l'énoncé ne fonctionnait pas.

## Partie 3 : Optimisation de la compression

### Ex 3.1

In [23]:
def arbre2(x,y,largeur,hauteur,l,seuil,i=0):
    a,b,d,e,flag=homogene(x,y,largeur,hauteur,seuil)
    if e<=seuil:
           l.append([x,y,largeur,hauteur,[int(a),int(b),int(d)],i,3])
    else :
           a1,b1,d1,e1,flag1 = homogene(x,y,largeur//2,hauteur,seuil)
           a2,b2,d2,e2,flag2 = homogene(x+largeur//2,y,largeur//2+largeur%2,hauteur,seuil)
           a3,b3,d3,e3,flag3 = homogene(x,y,largeur,hauteur//2,seuil)
           a4,b4,d4,e4,flag4 = homogene(x,y+hauteur//2,largeur,hauteur//2+hauteur%2,seuil)
            
           aa1,bb1,dd1,ee1,fflag1 = homogene(x,y,largeur//2,hauteur//2,seuil)
           aa2,bb2,dd2,ee2,fflag2 = homogene(x+largeur//2,y,largeur//2+largeur%2,hauteur//2,seuil)
           aa3,bb3,dd3,ee3,fflag3 = homogene(x,y+hauteur//2,largeur//2,hauteur//2,seuil)
           aa4,bb4,dd4,ee4,fflag4 = homogene(x+largeur//2,y+hauteur//2,largeur//2+largeur%2,hauteur//2+hauteur%2,seuil)
            
           if e1<=seuil and e2<=seuil:
                l.append([x,y,largeur,hauteur,[[a1,b1,d1],[a2,b2,d2]],i,1])
                
           elif e3<=seuil and e4<=seuil:
                l.append([x,y,largeur,hauteur,[[a3,b3,d3],[a4,b4,d4]],i,2])
                
           elif e3<=seuil and ee3<=seuil and ee4<=seuil:
                l.append([x,y,largeur,hauteur,[[a3,b2,d3],[aa3,bb3,dd3],[aa4,bb4,dd4]],i,4])
                
           elif ee1<=seuil and ee2<=seuil and e4<=seuil:
                l.append([x,y,largeur,hauteur,[[aa1,bb1,dd1],[aa2,bb2,dd2],[a4,b4,d4]],i,5])
                
           elif ee1<=seuil and e2<=seuil and ee3<=seuil:
                l.append([x,y,largeur,hauteur,[[aa1,bb1,dd1],[a2,b2,d2],[aa3,bb3,dd3]],i,6])
                
           elif e1<=seuil and ee2<=seuil and ee4<=seuil:
                l.append([x,y,largeur,hauteur,[[a1,b1,d1],[aa2,bb2,dd2],[aa4,bb4,dd4]],i,7])
                
           else : 
            a1,b1,d1,e1,flag1 = homogene(x,y,largeur//4,hauteur,seuil)
            a2,b2,d2,e2,flag2 = homogene(x+largeur//4,y,largeur//4,hauteur,seuil)
            a3,b3,d3,e3,flag3 = homogene(x+(2*largeur//4),y,largeur//4,hauteur,seuil)
            a4,b4,d4,e4,flag4 = homogene(x+(3*largeur//4),y,largeur//4,hauteur,seuil)
                
            if e1<=seuil and e2<=seuil and e3<=seuil and e4<=seuil:
                l.append([x,y,largeur,hauteur,[[a1,b1,d1],[a2,b2,d2],[a3,b3,d3],[a4,b4,d4]],i,8])
                    
            else :
                if largeur<4 or hauteur<4:
                    l.append([x,y,largeur,hauteur,None,i,0])
                    arbre2(x,y,largeur//2,hauteur//2,l,seuil,i+1)
                    arbre2(int(x+largeur//2),y,largeur//2+largeur%2,hauteur//2,l,seuil,i+1)
                    arbre2(x,int(y+hauteur//2),largeur//2,hauteur//2+hauteur%2,l,seuil,i+1)
                    arbre2(int(x+largeur//2),int(y+hauteur//2),largeur//2+largeur%2,hauteur//2+hauteur%2,l,seuil,i+1)
                    
                else:
                    a1,b1,d1,e1,flag1 = homogene(x,y,largeur,hauteur//4,seuil)
                    a2,b2,d2,e2,flag2 = homogene(x,y+hauteur//4,largeur,hauteur//4,seuil)
                    a3,b3,d3,e3,flag3 = homogene(x,y+2*hauteur//4,largeur,hauteur//4,seuil)
                    a4,b4,d4,e4,flag4 = homogene(x,y+3*hauteur//4,largeur,hauteur//4,seuil)
                        
                if e1<=seuil and e2<=seuil and e3<=seuil and e4<=seuil:
                    l.append([x,y,largeur,hauteur,[[a1,b1,d1],[a2,b2,d2],[a3,b3,d3],[a4,b4,d4]],i,9])
                            
                else :
                    l.append([x,y,largeur,hauteur,None,i,0])
                    arbre2(x,y,largeur//2,hauteur//2,l,seuil,i+1)
                    arbre2(int(x+largeur//2),y,largeur//2+largeur%2,hauteur//2,l,seuil,i+1)
                    arbre2(x,int(y+hauteur//2),largeur//2,hauteur//2+hauteur%2,l,seuil,i+1)
                    arbre2(int(x+largeur//2),int(y+hauteur//2),largeur//2+largeur%2,hauteur//2+hauteur%2,l,seuil,i+1)
               

Nous avons voulu optimiser la construction de cet arbre. Pour cela, nous commençons par tester si le noeud est du type 3 (un simple carré).
Si ce n’est pas la cas, nous testons pour savoir si le noeud peut se diviser en 3 parties.
Ensuite seulement, nous testons pour voir s’il peut se diviser en 4 bandes horizontales ou verticales.
Si aucun de ces cas n’est vérifié, il y alors a quadripartition.
Toutes les feuilles sont stockées dans la liste qui sera traitée plus tard.

In [24]:
T=[]
arbre2(0,0,W,H,T,seuil)

In [27]:
print(T) #la liste T stocke les feuilles

[[0, 0, 848, 565, None, 0, 0], [0, 0, 424, 282, None, 1, 0], [0, 0, 212, 141, None, 2, 0], [0, 0, 106, 70, None, 3, 0], [0, 0, 53, 35, None, 4, 0], [0, 0, 26, 17, None, 5, 0], [0, 0, 13, 8, [32, 36, 35], 6, 3], [13, 0, 13, 8, [31, 36, 35], 6, 3], [0, 8, 13, 9, [32, 36, 39], 6, 3], [13, 8, 13, 9, None, 6, 0], [13, 8, 6, 4, [32, 37, 35], 7, 3], [19, 8, 7, 4, [39, 45, 42], 7, 3], [13, 12, 6, 5, [[34, 61, 39], [37, 40, 45], [71, 73, 75]], 7, 4], [19, 12, 7, 5, None, 7, 0], [19, 12, 3, 2, [48, 54, 54], 8, 3], [22, 12, 4, 2, [72, 77, 82], 8, 3], [19, 14, 3, 3, [[75, 86, 75], [75, 75, 75], [87, 92, 98]], 8, 4], [22, 14, 4, 3, [[75, 75, 75], [110, 114, 124]], 8, 2], [26, 0, 27, 17, None, 5, 0], [26, 0, 13, 8, None, 6, 0], [26, 0, 6, 4, [31, 37, 36], 7, 3], [32, 0, 7, 4, [32, 37, 34], 7, 3], [26, 4, 6, 4, [34, 40, 38], 7, 3], [32, 4, 7, 4, [[38, 62, 45], [47, 57, 57], [63, 73, 73]], 7, 4], [39, 0, 14, 8, None, 6, 0], [39, 0, 7, 4, [41, 45, 47], 7, 3], [46, 0, 7, 4, None, 7, 0], [46, 0, 3, 2, [5

### Ex 3.2

In [28]:
def impression_feuille(T):   
    l1=[]
    i=0
    for i in range(len(T)):
        if T[i][4]!=None:
            x=T[i][0]
            y=T[i][1]
            largeur=T[i][2]
            hauteur=T[i][3]
            typ=T[i][6]
            if typ==1:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1]])
            if typ==2:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1]])
            if typ==3:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0]])
            if typ==4:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1],T[i][4][2]])
            if typ==5:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1],T[i][4][2]])
            if typ==6:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1],T[i][4][2]])
            if typ==7:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1],T[i][4][2]])
            if typ==8:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1],T[i][4][2],T[i][4][3]])
            if typ==9:
                l1.append([typ,x,y,largeur,hauteur,T[i][4][0],T[i][4][1],T[i][4][2],T[i][4][3]])
    return l1
 





On affiche dans une nouvelle liste chaque feuille avec son type, sa région couverte et la liste de ses couleurs. Nous n'avons pas réussi à créer une fonction récursive.

In [29]:
impression_feuille(T) #type au début, puis région couverte puis liste de ses couleurs

[[3, 0, 0, 13, 8, 32],
 [3, 13, 0, 13, 8, 31],
 [3, 0, 8, 13, 9, 32],
 [3, 13, 8, 6, 4, 32],
 [3, 19, 8, 7, 4, 39],
 [4, 13, 12, 6, 5, [34, 61, 39], [37, 40, 45], [71, 73, 75]],
 [3, 19, 12, 3, 2, 48],
 [3, 22, 12, 4, 2, 72],
 [4, 19, 14, 3, 3, [75, 86, 75], [75, 75, 75], [87, 92, 98]],
 [2, 22, 14, 4, 3, [75, 75, 75], [110, 114, 124]],
 [3, 26, 0, 6, 4, 31],
 [3, 32, 0, 7, 4, 32],
 [3, 26, 4, 6, 4, 34],
 [4, 32, 4, 7, 4, [38, 62, 45], [47, 57, 57], [63, 73, 73]],
 [3, 39, 0, 7, 4, 41],
 [3, 46, 0, 3, 2, 55],
 [3, 49, 0, 4, 2, 73],
 [3, 46, 2, 3, 2, 71],
 [3, 49, 2, 4, 2, 91],
 [3, 39, 4, 3, 2, 53],
 [3, 42, 4, 4, 2, 72],
 [3, 39, 6, 3, 2, 79],
 [3, 42, 6, 4, 2, 98],
 [5, 46, 4, 7, 4, [93, 100, 104], [111, 114, 121], [123, 128, 135]],
 [9, 26, 8, 6, 4, [49, 55, 55], [62, 67, 69], [72, 77, 81], [83, 87, 95]],
 [9,
  32,
  8,
  7,
  4,
  [76, 86, 87],
  [89, 99, 102],
  [101, 111, 114],
  [113, 120, 127]],
 [3, 26, 12, 3, 2, 95],
 [3, 29, 12, 3, 2, 112],
 [4, 26, 14, 3, 3, [75, 118, 75],

In [30]:
N=10000
n=[]

def arbre2compteur(x,y,largeur,hauteur,l,n,i=0):
    if len(n)<=N:
        a,b,d,e=homogene(x,y,largeur,hauteur)
        if e>=seuil:
            l.append([x,y,largeur,hauteur,[int(a),int(b),int(d)],i,3])
        else :
            a1,b1,d1,e1 = homogene(x,y,largeur//2,hauteur)
            a2,b2,d2,e2 = homogene(x+largeur//2,y,largeur//2+largeur%2,hauteur)
            a3,b3,d3,e3 = homogene(x,y,largeur,hauteur//2)
            a4,b4,d4,e4 = homogene(x,y+hauteur//2,largeur,hauteur//2+hauteur%2)
                
            aa1,bb1,dd1,ee1 = homogene(x,y,largeur//2,hauteur//2)
            aa2,bb2,dd2,ee2 = homogene(x+largeur//2,y,largeur//2+largeur%2,hauteur//2)
            aa3,bb3,dd3,ee3 = homogene(x,y+hauteur//2,largeur//2,hauteur//2)
            aa4,bb4,dd4,ee4 = homogene(x+largeur//2,y+hauteur//2,largeur//2+largeur%2,hauteur//2+hauteur%2)
                
            if e1<=seuil and e2<=seuil:
                l.append([x,y,largeur,hauteur,[[a1,b1,d1],[a2,b2,d2]],i,1])
                    
            elif e3<=seuil and e4 <=seuil:
                l.append([x,y,largeur,hauteur,[[a3,b3,d3],[a4,b4,d4]],i,2])
                    
            elif e3<=seuil and ee3<=seuil and ee4<=seuil:
                l.append([x,y,largeur,hauteur,[[a3,b2,d3],[aa3,bb3,dd3],[aa4,bb4,dd4]],i,4])
                    
            elif ee1<=seuil and ee2<=seuil and e4<=seuil:
                l.append([x,y,largeur,hauteur,[[aa1,bb1,dd1],[aa2,bb2,dd2],[a4,b4,d4]],i,5])
                    
            elif ee1<=seuil and e2<=seuil and ee3<=seuil:
                l.append([x,y,largeur,hauteur,[[aa1,bb1,dd1],[a2,b2,d2],[aa3,bb3,dd3]],i,6])
                    
            elif e1<=seuil and ee2<=seuil and ee4<=seuil:
                l.append([x,y,largeur,hauteur,[[a1,b1,d1],[aa2,bb2,dd2],[aa4,bb4,dd4]],i,7])
                    
            else : 
                a1,b1,d1,e1 = homogene(x,y,largeur//4,hauteur)
                a2,b2,d2,e2 = homogene(x+largeur//4,y,largeur//4,hauteur)
                a3,b3,d3,e3 = homogene(x+(2*largeur//4),y,largeur//4,hauteur)
                a4,b4,d4,e4 = homogene(x+(3*largeur//4),y,largeur//4,hauteur)
                    
                    
                if e1<=seuil and e2<=seuil and e3<=seuil and e4<=seuil:
                    l.append([x,y,largeur,hauteur,[[a1,b1,d1],[a2,b2,d2],[a3,b3,d3],[a4,b4,d4]],i,8])
                        
                else :
                    if largeur<4 or hauteur<4:
                        l.append([x,y,largeur,hauteur,None,i,0])
                        arbre2(x,y,largeur//2,hauteur//2,l,n.append(None),i+1)
                        arbre2(int(x+largeur//2),y,largeur//2+largeur%2,hauteur//2,l,n.append(None),i+1)
                        arbre2(x,int(y+hauteur//2),largeur//2,hauteur//2+hauteur%2,l,n.append(None),i+1)
                        arbre2(int(x+largeur//2),int(y+hauteur//2),largeur//2+largeur%2,hauteur//2+hauteur%2,l,n.append(None),i+1)
                        
                    else:
                            
                        a1,b1,d1,e1 = homogene(x,y,largeur,hauteur//4)
                        a2,b2,d2,e2 = homogene(x,y+hauteur//4,largeur,hauteur//4)
                        a3,b3,d3,e3 = homogene(x,y+2*hauteur//4,largeur,hauteur//4)
                        a4,b4,d4,e4 = homogene(x,y+3*hauteur//4,largeur,hauteur//4)
                            #la meme que ligne 194 mais avec hauteur
                        if e1<=seuil and e2<=seuil and e3<=seuil and e4<=seuil:
                            l.append([x,y,largeur,hauteur,[[a1,b1,d1],[a2,b2,d2],[a3,b3,d3],[a4,b4,d4]],i,9])
                                
                        else :
                            l.append([x,y,largeur,hauteur,None,i,0])
                            arbre2(x,y,largeur//2,hauteur//2,l,n.append(None),i+1)
                            arbre2(int(x+largeur//2),y,largeur//2+largeur%2,hauteur//2,l,n.append(None),i+1)
                            arbre2(x,int(y+hauteur//2),largeur//2,hauteur//2+hauteur%2,l,n.append(None),i+1)
                            arbre2(int(x+largeur//2),int(y+hauteur//2),largeur//2+largeur%2,hauteur//2+hauteur%2,l,n.append(None),i+1)
    
    else:
        a,b,d,e=homogene(x,y,largeur,hauteur)
        l.append([x,y,largeur,hauteur,[int(a),int(b),int(d)],i,3])
                
                

C'est la même chose que pour l'exercice 2 : il faudrait faire un parcours en largeur plutôt qu'en profondeur ou trouver une valeur pour le seuil optimal.

### Ex 3.4

Les yeux sont plus ou moins sensibles à des
couleurs (cf image ici il s’agit de la sensibilité des
bâtonnets). L’idée ici est de pondérer les couleurs par
des gaussiennes (3 gaussiennes avec les pic sur les
couleurs correspondantes). On détermine pour chaque
pic la valeur de R,V et B : cela nous donne M_r, M_v et M_b puis
nous estimons l’écart-type Ec_r, Ec_v et Ec_b. Puis pour chaque
couleur nous pondérons par : 
(Cst*((1/3)*exp((x-mr1)/sqrt(2)er1)^2)+
(1/3)*exp((x-mr2)/sqrt(2)er2)^2)+
(1/3)*exp((x-mr3)/sqrt(2)er1)^2)+Cst2).

<img src="couleur.png">

## Ex 3.5

La plateforme Eclope ne fonctionnant pas, nous n'avons pas pu estimer l'efficacité de notre code.